In [1]:
from train import parse_arguments
from data import PertDataloader, Network
import torch
import numpy as np
import scanpy as sc

args = parse_arguments()
if args['network_name'] == 'string':
    args['network_path'] = '/dfs/project/perturb-gnn/graphs/STRING_full_9606.csv'

if args['dataset'] == 'Norman2019':
    data_path = '/dfs/project/perturb-gnn/datasets/Norman2019_hvg+perts.h5ad'

adata = sc.read_h5ad(data_path)
if 'gene_symbols' not in adata.var.columns.values:
    adata.var['gene_symbols'] = adata.var['gene_name']
gene_list = [f for f in adata.var.gene_symbols.values]
args['gene_list'] = gene_list
args['num_genes'] = len(gene_list)

# Set up message passing network
network = Network(fname=args['network_path'], gene_list=args['gene_list'],
                  percentile=args['top_edge_percent'])

# Pertrubation dataloader
pertdl = PertDataloader(adata, network.G, network.weights, args)

There are 50506 edges in the PPI.
Creating pyg object for each cell in the data...
Local copy of pyg dataset is detected. Loading...
Loading splits...
Local copy of split is detected. Loading...
Creating dataloaders....
Dataloaders created...


In [2]:
def get_de(y_pred_gnn_8):
    y_pred_de = []
    for i in range(y_pred_gnn_8.shape[0]):
        if batch.pert!= 'ctrl':
            y_pred_de.append(y_pred_gnn_8[i][batch.de_idx[i]].detach().cpu().numpy())
            
    pred = np.array(y_pred_de)
    print(np.std(pred, axis = 0))
    return pred

In [3]:
for batch in pertdl.loaders['train_loader']:
    break

In [4]:
batch.to(args['device'])
pertdl.loaders['edge_index'] = pertdl.loaders['edge_index'].to(args['device'])
pertdl.loaders['edge_attr'] = pertdl.loaders['edge_attr'].to(args['device'])

In [16]:
# AE model
model = torch.load('./saved_models/AE_GCN_string_10.0_8_4_l2_Norman2019')
y_pred = model(batch, pertdl.loaders['edge_index'], pertdl.loaders['edge_attr'])
get_de(y_pred)

[0.8764108  1.1229603  0.7513373  0.8945958  0.83383197 0.790055
 0.87450427 0.9963516  1.4140565  1.1064742  1.2823851  1.3337672
 0.91394055 1.2040813  1.0342511  0.9800888  0.9436218  1.245337
 1.3299395  1.5215341 ]


array([[8.0420780e-01, 2.8497255e-01, 1.7038593e+00, ..., 1.2628704e-01,
        6.3101870e-01, 3.9187267e-02],
       [5.6415665e-01, 1.8657181e-02, 2.3147736e+00, ..., 2.9523345e-02,
        7.3317623e-01, 1.1090193e+00],
       [1.1964632e+00, 3.7265668e+00, 9.5379919e-02, ..., 4.2291431e+00,
        1.9854156e+00, 3.4998837e+00],
       ...,
       [1.1294720e+00, 8.0420780e-01, 1.0716734e+00, ..., 6.3101870e-01,
        8.8664019e-01, 3.9187267e-02],
       [3.7265668e+00, 2.1495759e-01, 4.8208743e-01, ..., 2.1891665e-02,
        4.7481060e-01, 1.9865072e+00],
       [9.6538599e-09, 9.6538599e-09, 9.6538599e-09, ..., 9.6538599e-09,
        9.6538599e-09, 9.6538599e-09]], dtype=float32)

In [17]:
# GNN+AE
model_gnn_ae = torch.load('./saved_models/GNN_Disentangle_AE_GCN_string_10.0_8_4_l2_Norman2019')
y_pred_gnn_ae = model_gnn_ae(batch, pertdl.loaders['edge_index'], pertdl.loaders['edge_attr'])
get_de(y_pred_gnn_ae)

[0.8737736  1.1408033  0.7699592  0.90710765 0.85020655 0.8017472
 0.8818852  1.0059756  1.3859051  1.1183817  1.2953357  1.3146404
 0.93745583 1.2257206  1.0556599  1.0106255  0.96838623 1.2594353
 1.335213   1.5433407 ]


array([[ 7.8680623e-01,  3.1867823e-01,  1.8298802e+00, ...,
         1.7220065e-01,  6.7904770e-01,  1.1251904e-01],
       [ 5.2956820e-01,  1.4672452e-02,  2.3447838e+00, ...,
         3.8809290e-03,  6.9440180e-01,  1.1379963e+00],
       [ 1.1108674e+00,  3.7906678e+00,  6.0987152e-02, ...,
         4.3300190e+00,  2.0620804e+00,  3.5862079e+00],
       ...,
       [ 1.0934349e+00,  7.8680623e-01,  1.1113191e+00, ...,
         6.7904770e-01,  1.0256479e+00,  1.1251904e-01],
       [ 3.7906678e+00,  8.1144407e-02,  4.4889605e-01, ...,
        -6.0287090e-03,  5.0768942e-01,  1.8864584e+00],
       [ 8.2764426e-09,  8.2764426e-09,  8.2764426e-09, ...,
         8.2764426e-09,  8.2764426e-09,  8.2764426e-09]], dtype=float32)

In [19]:
# GNN+AE+Old Model
model_gnn_ae_old = torch.load('./saved_models/GNN_AE_GCN_string_10.0_8_4_l2_Norman2019')
y_pred_gnn_ae_old = model_gnn_ae_old(batch, pertdl.loaders['edge_index'], pertdl.loaders['edge_attr'])
get_de(y_pred_gnn_ae)

[0.8737736  1.1408033  0.7699592  0.90710765 0.85020655 0.8017472
 0.8818852  1.0059756  1.3859051  1.1183817  1.2953357  1.3146404
 0.93745583 1.2257206  1.0556599  1.0106255  0.96838623 1.2594353
 1.335213   1.5433407 ]


array([[ 7.8680623e-01,  3.1867823e-01,  1.8298802e+00, ...,
         1.7220065e-01,  6.7904770e-01,  1.1251904e-01],
       [ 5.2956820e-01,  1.4672452e-02,  2.3447838e+00, ...,
         3.8809290e-03,  6.9440180e-01,  1.1379963e+00],
       [ 1.1108674e+00,  3.7906678e+00,  6.0987152e-02, ...,
         4.3300190e+00,  2.0620804e+00,  3.5862079e+00],
       ...,
       [ 1.0934349e+00,  7.8680623e-01,  1.1113191e+00, ...,
         6.7904770e-01,  1.0256479e+00,  1.1251904e-01],
       [ 3.7906678e+00,  8.1144407e-02,  4.4889605e-01, ...,
        -6.0287090e-03,  5.0768942e-01,  1.8864584e+00],
       [ 8.2764426e-09,  8.2764426e-09,  8.2764426e-09, ...,
         8.2764426e-09,  8.2764426e-09,  8.2764426e-09]], dtype=float32)

# We see that AE model overrides GNN => same result!
# With the GNN input embedding => all become diverse!

In [20]:
model_gnn = torch.load('./saved_models/GNN_Disentangle_GCN_string_10.0_8_4_l2_Norman2019_gene_emb')
y_pred_gnn = model_gnn(batch, pertdl.loaders['edge_index'], pertdl.loaders['edge_attr'])
get_de(y_pred_gnn)

[0.9000423  1.169048   0.77158016 0.92587966 0.89423466 0.82434374
 0.90947795 1.0445205  1.4756787  1.1515685  1.3317184  1.3586644
 0.9390059  1.2498214  1.0712336  1.0346953  0.96195704 1.2934763
 1.3659601  1.5759692 ]


array([[5.01635611e-01, 4.84946966e-01, 1.23146307e+00, ...,
        1.06709845e-01, 8.12832355e-01, 3.91360968e-01],
       [6.21215999e-01, 2.93259323e-03, 2.37223935e+00, ...,
        5.55051416e-02, 6.74661040e-01, 1.08783019e+00],
       [1.26829600e+00, 3.90747213e+00, 4.68779802e-02, ...,
        4.41748905e+00, 1.99936748e+00, 3.64192390e+00],
       ...,
       [1.09328866e+00, 8.21167469e-01, 1.19579089e+00, ...,
        7.33058095e-01, 1.30434990e+00, 8.33515525e-02],
       [3.94820285e+00, 1.88137755e-01, 4.63051260e-01, ...,
        5.51560521e-03, 3.84360254e-01, 1.96680117e+00],
       [1.24973059e-02, 1.24973059e-02, 1.24973059e-02, ...,
        1.24973059e-02, 1.24973059e-02, 1.24973059e-02]], dtype=float32)

In [21]:
model_gnn_8 = torch.load('./saved_models/GNN_Disentangle_GCN_string_10.0_8_10_l2_Norman2019_gene_emb')
y_pred_gnn_8 = model_gnn_8(batch, pertdl.loaders['edge_index'], pertdl.loaders['edge_attr'])
get_de(y_pred_gnn_8)

[0.90406585 1.1846099  0.7902668  0.9334528  0.8661452  0.830271
 0.9198453  1.0579336  1.4955195  1.1627995  1.3499572  1.3990538
 0.9804671  1.2891349  1.1151423  1.0553674  0.9894186  1.3141575
 1.3869747  1.5911956 ]


array([[ 7.6143795e-01,  4.2713875e-01,  1.2792203e+00, ...,
         7.9365194e-02,  8.5940063e-01,  3.1948569e-01],
       [ 6.3661528e-01,  1.7465085e-02,  2.3759556e+00, ...,
        -3.6806017e-03,  8.0487710e-01,  1.0931540e+00],
       [ 1.2552574e+00,  3.8736022e+00,  8.1730686e-02, ...,
         4.4062810e+00,  2.0485203e+00,  3.6863225e+00],
       ...,
       [ 1.1674958e+00,  8.3757120e-01,  1.1523023e+00, ...,
         7.9018891e-01,  9.6748269e-01,  7.9606906e-02],
       [ 3.9350481e+00,  1.3616484e-01,  4.9732885e-01, ...,
         1.7647833e-02,  4.1602266e-01,  2.0807407e+00],
       [ 2.3402572e-03,  2.3402572e-03,  2.3402572e-03, ...,
         2.3402572e-03,  2.3402572e-03,  2.3402572e-03]], dtype=float32)

In [5]:
model_gnn_8 = torch.load('./saved_models/GNN_Disentangle_GCN_string_10.0_8_10_l2_Norman2019_gene_emb')
y_pred_gnn_8 = model_gnn_8(batch, pertdl.loaders['edge_index'], pertdl.loaders['edge_attr'])

[0.90406585 1.1846099  0.7902668  0.9334528  0.8661452  0.830271
 0.9198453  1.0579336  1.4955195  1.1627995  1.3499572  1.3990538
 0.9804671  1.2891349  1.1151423  1.0553674  0.9894186  1.3141575
 1.3869747  1.5911956 ]


array([[ 7.6143795e-01,  4.2713875e-01,  1.2792203e+00, ...,
         7.9365179e-02,  8.5940063e-01,  3.1948569e-01],
       [ 6.3661528e-01,  1.7465085e-02,  2.3759556e+00, ...,
        -3.6806017e-03,  8.0487710e-01,  1.0931540e+00],
       [ 1.2552574e+00,  3.8736022e+00,  8.1730686e-02, ...,
         4.4062810e+00,  2.0485203e+00,  3.6863225e+00],
       ...,
       [ 1.1674958e+00,  8.3757120e-01,  1.1523023e+00, ...,
         7.9018891e-01,  9.6748269e-01,  7.9606906e-02],
       [ 3.9350481e+00,  1.3616484e-01,  4.9732885e-01, ...,
         1.7647833e-02,  4.1602266e-01,  2.0807407e+00],
       [ 2.3402572e-03,  2.3402572e-03,  2.3402572e-03, ...,
         2.3402572e-03,  2.3402572e-03,  2.3402572e-03]], dtype=float32)

In [6]:
from inference import evaluate, compute_metrics

test_res = evaluate(pertdl.loaders['test_loader'],
                        pertdl.loaders['edge_index'],
                        pertdl.loaders['edge_attr'],model_gnn_8, args)

test_metrics, test_pert_res = compute_metrics(test_res)

/dfs/user/kexinh/miniconda3/envs/eee/lib/python3.8/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/dfs/user/kexinh/miniconda3/envs/eee/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/dfs/user/kexinh/miniconda3/envs/eee/lib/python3.8/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/dfs/user/kexinh/miniconda3/envs/eee/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [7]:
test_metrics

{'mse_macro': 0.042565838,
 'mse_de_macro': 0.43476453,
 'mae_macro': 0.07234968,
 'mae_de_macro': 0.39245045,
 'spearman_macro': 0.4395219787672522,
 'spearman_de_macro': 0.7863154110526053,
 'pearson_macro': 0.8880746841008536,
 'pearson_de_macro': 0.876339334995341,
 'r2_macro': 0.7342619359981103,
 'r2_de_macro': 0.6952423794836438,
 'mse': 0.0028533,
 'mse_de': 0.18671036,
 'mae': 0.019279012,
 'mae_de': 0.2839504,
 'spearman': 0.7151998240735705,
 'spearman_de': 0.6896375647469604,
 'pearson': 0.9912412318750354,
 'pearson_de': 0.8345297850953829,
 'r2': 0.9821480811980519,
 'r2_de': 0.34559334278470527}

In [13]:
class No_Perturb(torch.nn.Module):
    """
    No Perturbation
    """

    def __init__(self):
        super(No_Perturb, self).__init__()        

    def forward(self, data, g, w):
        
        x = data.x
        x = x[:, 0].reshape(*data.y.shape)
        
        return x

In [14]:
np_model = No_Perturb()

In [15]:
np_model(batch, pertdl.loaders['edge_index'], pertdl.loaders['edge_attr'])

tensor([[0.0000, 0.0000, 0.0000,  ..., 3.3928, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 3.6719, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 3.4008, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 3.5169, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 3.5910, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 3.7759, 0.0000, 0.0000]],
       device='cuda:0')

In [12]:
y_pred_gnn_8.shape

torch.Size([100, 5045])